##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hub with Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/r1/tutorials/images/hub_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/r1/tutorials/images/hub_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

[TensorFlow Hub](http://tensorflow.org/hub) は事前学習済みモデルのコンポーネントを共有する一つの方法です。事前学習済みモデルの検索可能なリストは  [TensorFlow Module Hub](https://tfhub.dev/) をご覧ください。

このチュートリアルでは以下の事を説明します。

1. tf.keras での TensorFlow Hub の使い方
1. TensorFlow Hub を使って画像を分類する方法
1. シンプルな転移学習の方法

## セットアップ

### インポート

In [0]:
!pip install -U tensorflow_hub


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt

import tensorflow as tf
tf.enable_eager_execution()

In [0]:
import tensorflow_hub as hub

from tensorflow.keras import layers

## ImageNet 分類器

### 分類器をダウンロードする

mobilenet をロードするには hub.module を、それを keras レイヤーとしてまとめるには tf.keras.layers.Lambda を使ってください。

tfhub.dev にある [TensorFlow 1.x の画像分類器 URL](https://tfhub.dev/s?module-type=image-classification) であればすべてここで動作します。

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### 単一の画像で実行する

モデルを試すために単一の画像をダウンロードしてください。

In [0]:
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [0]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

バッチ次元を一つ追加し、画像をモデルに渡してください。

In [0]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

結果は 1001 の要素をもつロジットベクトルで、画像がそれぞれのクラスに属する確率を表します。

そのため、もっとも確率の高いクラスの ID は argmax でみつけることができます。

In [0]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

### 予測結果をデコードする

予測されたクラスの ID を ImageNet のラベルと突き合わせて、予測結果をデコードします。

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [0]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## シンプルな転移学習

TF Hub を利用すると、私たちが用意したデータセット内のクラスを認識するために、モデルの最上位層を再学習する事が容易になります。

### データセット

この例では、TensorFlow の花データセットを使用します。

In [0]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

このデータをモデルに読み込むもっとも簡単な方法は、 tf.keras.preprocessing.image.ImageDataGenerator を使用することです。

TensorFlow Hub のすべての画像モジュールは、  [0, 1] の範囲の float 入力を想定しています。これを実現するには ImageDataGenerator の rescale パラメータを使用してください。

画像のサイズについては後で処理されます。

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE)

結果のオブジェクトは、image_batch と label_batch のペアを返すイテレータです。

In [0]:
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

### 画像のバッチに対して分類器を実行する

それでは、画像のバッチで分類器を実行します。

In [0]:
result_batch = classifier.predict(image_batch)
result_batch.shape

In [0]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

これらの予測結果と実際の画像がどの程度一致しているか確認してください。

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

画像の属性については、 LICENSE.txt ファイルを参照してください。

結果は完全には程遠いですが、これらがこのモデルのために学習されたクラスでない事を考えると悪くはないです（ 「デイジー」を除く）。


### ヘッドレスモデルをダウンロードする

TensorFlow Hub はさらに、最上位の分類層がないモデルを配布することもしています。これらは転移学習を簡単に行うために使用することができます。

tfhub.dev からの [TensorFlow 1.x の画像特徴ベクトルの URL](https://tfhub.dev/s?module-type=image-feature-vector) はすべてここで動作します。

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

モジュールを作成し、入力として期待される画像サイズを確認します。

In [0]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))

特徴抽出器は、各画像に対して 1280 要素のベクトルを返します。

In [0]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

学習が新しい分類層のみを修正するように、特徴抽出層の変数を freeze（固定）します。

In [0]:
feature_extractor_layer.trainable = False

### 分類ヘッドを追加する

hub レイヤーを tf.keras.Sequential モデルでラップし、新しい分類層を一つ追加します。

In [0]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes, activation='softmax')
])

model.summary()

In [0]:
predictions = model(image_batch)

In [0]:
predictions.shape

### モデルを学習する

学習プロセスを構成するために、 compile を使用してください。

In [0]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

今度は .fit メソッドを使ってモデルを学習します。

この例を短くするために、2エポックだけ学習を実行します。学習の進行状況を可視化するには、エポック平均ではなく、カスタムコールバックを使用して各バッチの損失と精度を個別に記録します。

In [0]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [0]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit(image_data, epochs=2,
                    steps_per_epoch=steps_per_epoch,
                    callbacks = [batch_stats_callback])

これで、ほんの数回の学習の繰り返しでも、モデルがタスクを進めていることがわかったと思います。

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [0]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### 予測結果を確認する

前からプロットをやり直すには、まずクラス名の順序付きリストを取得します。

In [0]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

モデルを介してイメージバッチを実行し、インデックスをクラス名に変換します。

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

結果をプロットします。

In [0]:
label_id = np.argmax(label_batch, axis=-1)

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## モデルをエクスポートする

モデルの学習が完了したので、saved model としてエクスポートします。

In [0]:
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
tf.keras.experimental.export_saved_model(model, export_path)

export_path

それではモデルをリロードできることを確認してください。リロードしたモデルでもおなじ結果が得られます。

In [0]:
reloaded = tf.keras.experimental.load_from_saved_model(export_path, custom_objects={'KerasLayer':hub.KerasLayer})

In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
abs(reloaded_result_batch - result_batch).max()

この saved model は後で推論のためにロードするか、もしくは [TFLite](https://www.tensorflow.org/lite/convert/) か [TFjs](https://github.com/tensorflow/tfjs-converter).に変換することができます。